<a href="https://colab.research.google.com/github/Nobobi-Hasan/PointNeXt-PartSegmentation-FallenTrees/blob/main/PointNeXt_03_05_Testing_shapenetpart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
import shutil
import torch
import glob
import subprocess
import sys

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Project Paths
DRIVE_PROJECT_ROOT = "/content/drive/MyDrive/ML_Projects/PointNeXt"
DRIVE_DATA_ZIP = os.path.join(DRIVE_PROJECT_ROOT, "Data/processed_data.zip")
DRIVE_RAW_DATA_PATH = os.path.join(DRIVE_PROJECT_ROOT, "Data/data.zip")
DRIVE_MODEL_DIR = os.path.join(DRIVE_PROJECT_ROOT, "Models")
# raw_extract_path = "/content/raw_data"

# Find the latest best model automatically
model_candidates = glob.glob(os.path.join(DRIVE_MODEL_DIR, "*_best.pth"))
if not model_candidates:
    print("\u274c Error: No model found in Drive!")
    model_path = ""
else:
    # Pick the most recently created file
    model_path = max(model_candidates, key=os.path.getmtime)
    print(f"\u2705 Found Model: {os.path.basename(model_path)}")

print(f"Project Root: {DRIVE_PROJECT_ROOT}")

✅ Found Model: shapenetpart-train-custom_fallen_trees-ngpus1-seed7520-20251212-155340-E84zcZ3REWTnK2wbniefub_ckpt_best.pth
Project Root: /content/drive/MyDrive/ML_Projects/PointNeXt


# Re-Install Environment (Identical to Training)

In [4]:
# Copy & Unzip Processed Data
if not os.path.exists("/content/processed_data"):
    if os.path.exists(DRIVE_DATA_ZIP):
        print("Copying processed data from Drive...")
        shutil.copy(DRIVE_DATA_ZIP, "/content/processed_data.zip")
        print("Unzipping...")
        !unzip -q -o /content/processed_data.zip -d /
        print("Data Ready.")
    else:
        print(f"Error: Processed Data zip not found at {DRIVE_DATA_ZIP}")

In [5]:
# Copy & Unzip Raw Data
if not os.path.exists("/content/raw_data"):
    if os.path.exists(DRIVE_RAW_DATA_PATH):
        print("Copying raw data from Drive...")
        shutil.copy(DRIVE_RAW_DATA_PATH, "/content/raw_data.zip")
        print("Unzipping...")
        !unzip -q /content/raw_data.zip -d /content/raw_data
        print("Data Ready.")
    else:
        print(f"Error: Raw Data zip not found at {DRIVE_RAW_DATA_PATH}")

In [6]:
# remove __MACOSX from raw_data folder
!rm -rf /content/raw_data/__MACOSX

In [7]:
# Clone PointNeXt (Clean)
if os.path.exists("/content/PointNeXt"):
    shutil.rmtree("/content/PointNeXt")
print("Cloning PointNeXt...")
%cd /content
!git clone https://github.com/guochengqian/PointNeXt.git

Cloning PointNeXt...
/content
Cloning into 'PointNeXt'...
remote: Enumerating objects: 723, done.
remote: Counting objects: 100% (327/327), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 723 (delta 247), reused 182 (delta 182), pack-reused 396 (from 1)
Receiving objects: 100% (723/723), 5.22 MiB | 14.34 MiB/s, done.
Resolving deltas: 100% (428/428), done.


In [8]:
# For openpoints

%cd /content/PointNeXt

# 1. Replace SSH url with HTTPS url in .gitmodules
!sed -i 's/git@github.com:/https:\/\/github.com\//' .gitmodules

# 2. Sync the new URL
!git submodule sync

# 3. Update the submodule (This will work now)
print("Downloading openpoints via HTTPS...")
!git submodule update --init --recursive

print("\u2705 Submodule 'openpoints' downloaded successfully.")

/content/PointNeXt
Submodule 'openpoints' (https://github.com/guochengqian/openpoints.git) registered for path 'openpoints'
Cloning into '/content/PointNeXt/openpoints'...
Submodule path 'openpoints': checked out 'baeca5e319aa2e756d179e494469eb7f5ffd29f0'
✅ Submodule 'openpoints' downloaded successfully.


In [9]:
# Install Dependencies
print("Installing Dependencies...")
%cd /content/PointNeXt

# A. Install PyTorch 2.4.0 (Compatible)
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121

# B. Install torch-scatter/sparse
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

# C. Fix requirements.txt
!sed -i 's/==.*//g' requirements.txt

# D. Install requirements
!pip install -r requirements.txt

# E. SKIP 'pip install -e .' (Because setup.py is missing) in training
!pip install multimethod shortuuid wandb deepspeed hjson ml_collections
# !pip install -e .

print("\u2705 Dependencies Installed. (Skipped setup.py)")

Installing Dependencies...
/content/PointNeXt
Looking in indexes: https://download.pytorch.org/whl/cu121
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
  Using cached pyvista-0.46.4-py3-none-any.whl.metadata (15 kB)
  Using cached deepspeed-0.18.3-py3-none-any.whl
  Using cached mkdocs_material-9.7.0-py3-none-any.whl.metadata (19 kB)
  Using cached mkdocs_awesome_pages_plugin-2.10.1-py3-none-any.whl.metadata (13 kB)
  Using cached vtk-9.5.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.6 kB)
  Using cached mkdocs-1.6.1-py3-none-any.whl.metadata (6.0 kB)
Using cached pyvista-0.46.4-py3-none-any.whl (2.4 MB)
Using cached mkdocs_material-9.7.0-py3-none-any.whl (9.3 MB)
Using cached mkdocs_awesome_pages_plugin-2.10.1-py3-none-any.whl (15 kB)
Using cached mkdocs-1.6.1-py3-none-any.whl (3.9 MB)
Using cached vtk-9.5.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (112.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 2.8 MB

# Compile C++ Kernels

In [10]:
# 1. Search for the missing setup.py
print("Searching for C++ kernel setup.py...")
target_dir = "/content/PointNeXt/openpoints/cpp"
found_setup = False

for root, dirs, files in os.walk(target_dir):
    if "setup.py" in files:
        print(f"\u2705 Found setup.py at: {root}")
        found_setup = True

        # 2. Force Compile
        print(f"Compiling kernels in {root}...")
        try:
            subprocess.check_call([sys.executable, "setup.py", "install"], cwd=root)
            print("Compilation Successful!")
        except subprocess.CalledProcessError as e:
            print(f"\u274c Compilation Failed: {e}")

if not found_setup:
    print("\u274c Critical Error: Could not find setup.py anywhere in openpoints/cpp!")
    print("Did the 'git submodule update' step finish successfully?")

Searching for C++ kernel setup.py...
✅ Found setup.py at: /content/PointNeXt/openpoints/cpp/pointnet2_batch
Compiling kernels in /content/PointNeXt/openpoints/cpp/pointnet2_batch...
Compilation Successful!
✅ Found setup.py at: /content/PointNeXt/openpoints/cpp/emd
Compiling kernels in /content/PointNeXt/openpoints/cpp/emd...
Compilation Successful!
✅ Found setup.py at: /content/PointNeXt/openpoints/cpp/pointops
Compiling kernels in /content/PointNeXt/openpoints/cpp/pointops...
Compilation Successful!
✅ Found setup.py at: /content/PointNeXt/openpoints/cpp/subsampling
Compiling kernels in /content/PointNeXt/openpoints/cpp/subsampling...
❌ Compilation Failed: Command '['/usr/bin/python3', 'setup.py', 'install']' returned non-zero exit status 1.
✅ Found setup.py at: /content/PointNeXt/openpoints/cpp/chamfer_dist
Compiling kernels in /content/PointNeXt/openpoints/cpp/chamfer_dist...
Compilation Successful!


# Re-Create Config File

In [12]:
config_path = "/content/PointNeXt/cfgs/shapenetpart/custom_fallen_trees.yaml"

config_content = """
num_classes: 4
shape_classes: 2
epochs: 100

# Explicitly define feature keys to match the custom data loader
feature_keys: 'pos,x'

model:
  NAME: BasePartSeg
  encoder_args:
    NAME: PointNextEncoder
    blocks: [1, 1, 1, 1, 1]
    strides: [1, 2, 2, 2, 2]
    width: 32
    in_channels: 7
    sa_layers: 3
    sa_use_res: True
    radius: 0.1
    radius_scaling: 2.5
    nsample: 32
    expansion: 4
    aggr_args:
      feature_type: 'dp_fj'
    reduction: 'max'
    group_args:
      NAME: 'ballquery'
      normalize_dp: True
    conv_args:
      order: conv-norm-act
    act_args:
      act: 'relu'
    norm_args:
      norm: 'bn'
  decoder_args:
    NAME: PointNextPartDecoder
    cls_map: curvenet
  cls_args:
    NAME: SegHead
    global_feat: max,avg
    num_classes: 4
    shape_classes: 2
    in_channels: null
    norm_args:
      norm: 'bn'

dataset:
  common:
    NAME: FallenTreePart
    data_root: /content/processed_data
    use_normal: False
    use_xyz: True
    num_points: 2048
  train:
    split: train
  val:
    split: val

batch_size: 16
dataloader:
  num_workers: 4

lr: 0.001
min_lr: null
optimizer:
  NAME: adamw
  weight_decay: 1.0e-4

criterion_args:
  NAME: Poly1FocalLoss

# scheduler
epochs: 100
sched: multistep
decay_epochs: [70, 90]
decay_rate: 0.1
warmup_epochs: 0

datatransforms:
  train: [PointsToTensor, PointCloudScaling, PointCloudCenterAndNormalize, PointCloudJitter, ChromaticDropGPU]
  val: [PointsToTensor, PointCloudCenterAndNormalize]
  kwargs:
    jitter_sigma: 0.001
    jitter_clip: 0.005
    scale: [0.8, 1.2]
    gravity_dim: 1
    angle: [0, 1.0, 0]

log_dir: /content/PointNeXt/log/shapenetpart/custom_trees
"""

with open(config_path, 'w') as f:
    f.write(config_content)
print(f"\u2705 Config Updated.")

✅ Config Updated.


# Re-Create Custom Dataset File (fallentree.py)

In [13]:
# Create the directory
new_dataset_dir = "/content/PointNeXt/openpoints/dataset/fallentree"
os.makedirs(new_dataset_dir, exist_ok=True)
print(f"\U0001F4C2 Created folder: {new_dataset_dir}")

# Create the '__init__.py' to make it a package
init_path = os.path.join(new_dataset_dir, "__init__.py")
with open(init_path, 'w') as f:
    f.write("from .fallentree import FallenTreePart\n")
print(f"\u2705 Created: {init_path}")

📂 Created folder: /content/PointNeXt/openpoints/dataset/fallentree
✅ Created: /content/PointNeXt/openpoints/dataset/fallentree/__init__.py


In [14]:
# Define Path
code_path = "/content/PointNeXt/openpoints/dataset/fallentree/fallentree.py"

# Define Code
dataset_code = """
import os
import glob
import json
import logging
import numpy as np
import torch
from torch.utils.data import Dataset
from ..build import DATASETS


# NEW QUOTA-BASED SAMPLING FUNCTION
def quota_sample(xyz, part_labels, npoint):
    \"\"\"
    For TRAINING: Forces small classes (Roots/Pits) to have a minimum number of points.
    This helps the model see them, even if they are tiny in the real tree.
    \"\"\"

    total_points = len(xyz)
    if total_points <= npoint:
        return np.random.choice(total_points, npoint, replace=True)

    unique_parts = np.unique(part_labels)

    # THE STRATEGY
    # We want to reserve specific slots for the hard classes.
    # Total 2048 points.
    # If Root exists, force 256 points.
    # If Pit exists, force 256 points.
    # The rest from all points.

    min_points_map = {
        0: 0,    # Standing Trunk: No minimum forced
        1: 0,    # Fallen Trunk: No minimum forced
        2: 256,  # Root: 256 points minimum
        3: 256   # Pit: 256 points minimum
    }

    indices_collected = []

    # 1. Collect the hard classes first
    for part in unique_parts:
        if part in min_points_map and min_points_map[part] > 0:
            part_indices = np.where(part_labels == part)[0]

            # If the part is smaller than the quota, take it all with replacement (duplicate points)
            req_count = min_points_map[part]

            if len(part_indices) >= req_count:
                chosen = np.random.choice(part_indices, req_count, replace=False)
            else:
                chosen = np.random.choice(part_indices, req_count, replace=True)

            indices_collected.extend(chosen)

    # Fill the rest of the 2048 points with points left
    current_count = len(indices_collected)
    remaining_slots = npoint - current_count

    if remaining_slots > 0:
        all_indices = np.arange(total_points)
        fill_indices = np.random.choice(all_indices, remaining_slots, replace=False)
        indices_collected.extend(fill_indices)

    final_indices = np.array(indices_collected)

    # Shuffle so the roots or pits aren't all at the beginning
    np.random.shuffle(final_indices)

    # If by some edge case we have too many, trim it
    if len(final_indices) > npoint:
        final_indices = final_indices[:npoint]

    return final_indices


@DATASETS.register_module()
class FallenTreePart(Dataset):
    classes = ['standing', 'fallen']
    num_classes = 4
    shape_classes = 2

    # --- FIX: Add dummy key -1 for part_seg_refinement compatibility ---
    # -1 points to all parts (0,1,2,3) so the code can find the max index
    cls2parts = {
        0: [0],
        1: [1, 2, 3],
        -1: [0, 1, 2, 3]
    }

    part_start = [0, 1]

    # Pre-compute embedding
    cls2partembed = torch.zeros(shape_classes, num_classes)
    for i in [0, 1]: # Iterate only real classes
        idx = cls2parts[i]
        cls2partembed[i].scatter_(0, torch.LongTensor(idx), 1)

    def __init__(self,
                 data_root,
                 split=None,
                 num_points=2048,
                 use_normal=False,
                 use_xyz=True,
                 **kwargs):
        self.root = data_root
        self.npoints = num_points
        self.split = split

        # if split == 'val': split_name = 'test'
        # else: split_name = split

        split_name = split

        split_file = os.path.join(self.root, 'train_test_split', f'shuffled_{split_name}_file_list.json')
        if not os.path.exists(split_file):
             raise FileNotFoundError(f"Split list not found: {split_file}")

        logging.info(f"Loading {split} split from: {split_file}")
        with open(split_file, 'r') as f:
            raw_list = json.load(f)

        self.file_list = []
        for item in raw_list:
            clean_item = item.replace(\"\\\\\", \"/\")
            fname = os.path.basename(clean_item)
            candidates = [
                clean_item,
                os.path.join(self.root, clean_item),
                os.path.join(self.root, '0', fname),
                os.path.join(self.root, '1', fname)
            ]
            found = False
            for path in candidates:
                if os.path.exists(path):
                    self.file_list.append(path)
                    found = True
                    break

        logging.info(f"Found {len(self.file_list)} valid files for {split} split.")

    def __getitem__(self, index):
        file_path = self.file_list[index]
        cls_idx = 0 if '/0/' in file_path.replace('\\\\', '/') else 1

        data = np.load(file_path).astype(np.float32)
        xyz = data[:, 0:3]
        features = data[:, 3:7]
        part_label = data[:, 7].astype(np.int64)
        cls_label = np.array([cls_idx]).astype(np.int64)

        if self.split == 'train':
            choice = quota_sample(xyz, part_label, self.npoints)
        else:
            if len(xyz) >= self.npoints:
                choice = np.random.choice(len(xyz), self.npoints, replace=False)
            else:
                choice = np.random.choice(len(xyz), self.npoints, replace=True)

        xyz = xyz[choice]
        features = features[choice]
        part_label = part_label[choice]

        return {'pos': xyz, 'x': features, 'y': part_label, 'cls': cls_label}

    def __len__(self):
        return len(self.file_list)
"""

with open(code_path, 'w') as f:
    f.write(dataset_code)
print(f"\u2705 Updated: {code_path}.")

✅ Updated: /content/PointNeXt/openpoints/dataset/fallentree/fallentree.py.


In [15]:
# Register the new folder in the Main Library
# We need to add "from .fallentree import FallenTreePart" to openpoints/dataset/__init__.py

main_init = "/content/PointNeXt/openpoints/dataset/__init__.py"
with open(main_init, 'r') as f:
    content = f.read()

if "fallentree" not in content:
    print("Registering new dataset in main __init__.py...")
    with open(main_init, 'a') as f:
        f.write("\nfrom .fallentree import FallenTreePart\n")
    print("\u2705 Dataset Class Restored.")
else:
    print("\u2705 Already registered.")

Registering new dataset in main __init__.py...
✅ Dataset Class Restored.


# Inference (Generates Output)

In [16]:
# Force Load C++ Extension
import sys
import os

# Add the specific build folder to Python path
cpp_root = "/content/PointNeXt/openpoints/cpp/pointnet2_batch"
build_dirs = [os.path.join(cpp_root, d) for d in os.listdir(cpp_root) if d.startswith('build')]

if build_dirs:
    # Find the lib folder inside build
    lib_dir = os.path.join(build_dirs[0], [d for d in os.listdir(build_dirs[0]) if d.startswith('lib')][0])
    if os.path.exists(lib_dir):
        print(f"Adding to path: {lib_dir}")
        sys.path.insert(0, lib_dir)
    else:
        print("Could not find lib folder in build.")
else:
    print("No build folder found. Did compilation actually finish?")

# Try importing manually to check
try:
    import pointnet2_batch_cuda
    print("\u2705 SUCCESS: pointnet2_batch_cuda is importable.")
except ImportError as e:
    print(f"\u274c STILL FAILING: {e}")
    print("Trying fallback: Re-running setup.py install in verbose mode...")
    # Last ditch effort: Run install again with verbose output to see where it goes
    %cd /content/PointNeXt/openpoints/cpp/pointnet2_batch
    !python setup.py install

Adding to path: /content/PointNeXt/openpoints/cpp/pointnet2_batch/build/lib.linux-x86_64-cpython-312
✅ SUCCESS: pointnet2_batch_cuda is importable.


In [17]:
# Force Load ALL C++ Extensions (Universal Fix)
import sys
import os
import importlib

cpp_base = "/content/PointNeXt/openpoints/cpp"
modules_to_fix = [
    "pointnet2_batch",
    "chamfer_dist",
    "emd",
    "pointops"
]

print("\U0001F527 Patching Python Path for C++ Modules...")

for module in modules_to_fix:
    mod_path = os.path.join(cpp_base, module)

    # Find the build/lib folder
    build_dirs = [os.path.join(mod_path, d) for d in os.listdir(mod_path) if d.startswith('build')]

    if build_dirs:
        # Find the lib folder inside build
        lib_content = [d for d in os.listdir(build_dirs[0]) if d.startswith('lib')]
        if lib_content:
            lib_dir = os.path.join(build_dirs[0], lib_content[0])

            if os.path.exists(lib_dir):
                if lib_dir not in sys.path:
                    sys.path.insert(0, lib_dir)
                    print(f"   \u2705 Added to path: {module} -> {lib_dir}")
                else:
                    print(f"   \u2705  Already in path: {module}")
            else:
                print(f"   \u26A0\uFE0F Lib folder exists but path is wrong: {lib_dir}")
        else:
             print(f"   \u26A0\uFE0F No 'lib' folder found in {build_dirs[0]}")
    else:
        print(f"   \u274c No 'build' folder found for {module}. Did compilation fail?")

print("\n\U0001F504 Verifying Imports...")
try:
    import pointnet2_batch_cuda
    print("   \u2705 pointnet2_batch_cuda: OK")
except ImportError as e: print(f"   \u274c pointnet2_batch_cuda: FAILED ({e})")

try:
    import chamfer
    print("   \u2705 chamfer: OK")
except ImportError as e: print(f"   \u274c chamfer: FAILED ({e})")

try:
    import emd
    print("   \u2705 emd: OK")
except ImportError as e: print(f"   \u274c emd: FAILED ({e})")

print("\n\U0001F680 Ready for Inference.")

🔧 Patching Python Path for C++ Modules...
   ✅  Already in path: pointnet2_batch
   ✅ Added to path: chamfer_dist -> /content/PointNeXt/openpoints/cpp/chamfer_dist/build/lib.linux-x86_64-cpython-312
   ✅ Added to path: emd -> /content/PointNeXt/openpoints/cpp/emd/build/lib.linux-x86_64-cpython-312
   ✅ Added to path: pointops -> /content/PointNeXt/openpoints/cpp/pointops/build/lib.linux-x86_64-cpython-312

🔄 Verifying Imports...
   ✅ pointnet2_batch_cuda: OK
   ✅ chamfer: OK
   ❌ emd: FAILED (No module named 'emd')

🚀 Ready for Inference.


In [18]:
import os
import sys

# Check the filename inside the build folder
emd_build_dir = "/content/PointNeXt/openpoints/cpp/emd/build/lib.linux-x86_64-cpython-312"

print(f"\U0001F4C2 Checking contents of: {emd_build_dir}")
if os.path.exists(emd_build_dir):
    files = os.listdir(emd_build_dir)
    print(f"   Found files: {files}")

    # Add to path if not there
    if emd_build_dir not in sys.path:
        sys.path.insert(0, emd_build_dir)

    # Try importing based on filename
    for f in files:
        if f.endswith(".so"):
            module_name = f.split(".")[0] # e.g., "emd_cuda" from "emd_cuda.cpython..."
            print(f"   Trying to import: '{module_name}'")
            try:
                __import__(module_name)
                print(f"   \u2705 SUCCESS! Imported '{module_name}'")
            except ImportError as e:
                print(f"   \u274c Failed to import '{module_name}': {e}")
else:
    print("\u274c Directory not found.")

📂 Checking contents of: /content/PointNeXt/openpoints/cpp/emd/build/lib.linux-x86_64-cpython-312
   Found files: ['emd_cuda.cpython-312-x86_64-linux-gnu.so']
   Trying to import: 'emd_cuda'
   ✅ SUCCESS! Imported 'emd_cuda'


In [19]:
def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
    pc = pc / m
    return pc

def rgb_normalize(points):
    min_vals = points.min(axis=0)      # shape: (,3)
    max_vals = points.max(axis=0)      # shape: (,3)

    normalized = (points - min_vals) / (max_vals - min_vals + 1e-8)  # avoid div by zero

    return normalized

def intensity_normalize(points):
    min_vals = points.min()      # shape: (,3)
    max_vals = points.max()      # shape: (,3)

    normalized = (points - min_vals) / (max_vals - min_vals + 1e-8)  # avoid div by zero

    return normalized

In [20]:
import torch
import numpy as np
import os
import json
from tqdm import tqdm
from openpoints.utils import EasyConfig
from openpoints.models import build_model_from_cfg
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

# DEFINE FPS FUNCTION
def farthest_point_sample(xyz, npoint):
    N, C = xyz.shape
    centroids = np.zeros((npoint,), dtype=np.int64)
    distance = np.ones((N,)) * 1e10
    farthest = np.random.randint(0, N)

    for i in range(npoint):
        centroids[i] = farthest
        centroid = xyz[farthest, :]
        dist = np.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance, -1)
    return centroids


TEST_LIST = "/content/processed_data/train_test_split/shuffled_test_file_list.json"
OUTPUT_DIR = "/content/inference_results_12ch"
RAW_DIR = "/content/raw_data"
NUM_POINTS = 2048
NUM_CLASSES = 4

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

print(f"\n\U0001F680 Starting Inference using model: {os.path.basename(model_path)}")

# 1. Load Config & Model
cfg = EasyConfig()
cfg.load(config_path, recursive=True)
model = build_model_from_cfg(cfg.model).cuda()
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model'])
model.eval()
print("\u2705 Model loaded.")

# 2. Load Test List
with open(TEST_LIST, 'r') as f:
    test_files = json.load(f)

# 3. Metrics Init
total_correct = 0
total_points = 0
instance_ious = []
global_confusion = np.zeros((NUM_CLASSES, NUM_CLASSES))

# 4. Inference Loop
for entry in tqdm(test_files):
    filename = os.path.basename(entry.replace("\\", "/"))

    # Find Raw File
    found_path = None
    for root, dirs, files in os.walk(RAW_DIR):
        if filename in files:
            found_path = os.path.join(root, filename)
            break

    if found_path:
        # A. Load Raw Data
        raw_data_11c = np.load(found_path).astype(np.float32)
        N_points = raw_data_11c.shape[0]

        # Get GT for metrics
        gt_labels = raw_data_11c[:, 10].astype(np.int64)

        # CLASS DETECTION
        # Use filename, not path (safer)
        if "standing" in filename:
            cls_idx = 0
        else:
            cls_idx = 1

        # B. NORMALIZATION
        # Extract
        xyz_raw = raw_data_11c[:, 0:3]
        rgb_raw = raw_data_11c[:, 3:6]
        int_raw = raw_data_11c[:, 6].reshape(-1, 1)

        # Normalize
        xyz_norm = pc_normalize(xyz_raw)
        rgb_norm = rgb_normalize(rgb_raw)
        int_norm = intensity_normalize(int_raw)

        # Combine Features (RGB + I)
        features_norm = np.hstack((rgb_norm, int_norm))

        # C. Resample
        if N_points >= NUM_POINTS:
            # choice = np.random.choice(N_points, NUM_POINTS, replace=False)
            choice = farthest_point_sample(xyz_norm, NUM_POINTS)
        else:
            # Upsample with random repeat if too small
            choice = np.random.choice(N_points, NUM_POINTS, replace=True)

        xyz_sampled = xyz_norm[choice]
        features_sampled = features_norm[choice]

        # D. Predict
        pos_tensor = torch.from_numpy(xyz_sampled).unsqueeze(0).cuda()
        # X expects (B, C, N) -> Transpose
        model_input = np.hstack((xyz_sampled, features_sampled)) # XYZ+RGB+I (7 channels)
        x_tensor = torch.from_numpy(model_input).unsqueeze(0).cuda().transpose(1, 2).contiguous()
        cls_tensor = torch.tensor([[cls_idx]], dtype=torch.long).cuda()

        with torch.no_grad():
            data_batch = {'pos': pos_tensor, 'x': x_tensor, 'cls': cls_tensor}
            logits = model(data_batch)
            preds_sampled = torch.argmax(logits, dim=1).squeeze(0).cpu().numpy()

        # E. Interpolate
        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(xyz_sampled, preds_sampled)
        preds_full = knn.predict(xyz_norm)

        # --- METRICS ---
        correct = np.sum(preds_full == gt_labels)
        total_correct += correct
        total_points += N_points

        # Mask invalid classes based on object type (Standing vs Fallen)
        # Standing (0) should ignore roots/pits (2, 3)
        valid_parts = [0] if cls_idx == 0 else [1, 2, 3]

        # Instance IoU
        union = np.histogram(preds_full, bins=NUM_CLASSES, range=(0, NUM_CLASSES))[0] + \
                np.histogram(gt_labels, bins=NUM_CLASSES, range=(0, NUM_CLASSES))[0]
        intersection = np.histogram(preds_full[preds_full == gt_labels], bins=NUM_CLASSES, range=(0, NUM_CLASSES))[0]
        union = union - intersection
        iou = intersection / (union + 1e-6)

        # Only average IoU for parts that SHOULD exist
        instance_ious.append(np.mean(iou[valid_parts]))

        # Global Confusion
        mask = (gt_labels >= 0) & (gt_labels < NUM_CLASSES)
        global_confusion += confusion_matrix(gt_labels[mask], preds_full[mask], labels=np.arange(NUM_CLASSES))

        # F. Save Result (N, 12)
        preds_col = preds_full.reshape(-1, 1)
        final_data = np.hstack((raw_data_11c, preds_col))

        save_name = filename.replace(".npy", "_ALL_PTS.npy")
        np.save(os.path.join(OUTPUT_DIR, save_name), final_data)

    else:
        print(f"\u26A0\ufe0f Warning: Could not find raw file {filename}")

# --- REPORT ---
OA = total_correct / total_points * 100
mIoU_Instance = np.mean(instance_ious) * 100

class_intersection = np.diag(global_confusion)
class_union = np.sum(global_confusion, axis=1) + np.sum(global_confusion, axis=0) - class_intersection
class_ious = class_intersection / (class_union + 1e-6) * 100
mIoU_Class = np.mean(class_ious)

print("\n" + "="*40)
print(f"Overall Accuracy (OA): {OA:.2f}%")
print(f"Instance mIoU:       {mIoU_Instance:.2f}%")
print(f"Class mIoU:          {mIoU_Class:.2f}%")
print("="*30)

print("PER-PART BREAKDOWN:")
part_names = ["Standing Trunk (0)", "Fallen Trunk (1)", "Root Ball (2)", "Pit (3)"]
for i, score in enumerate(class_ious):
    print(f"{part_names[i]:<20}: {score:.2f}%")
print("="*40)

# Zip
!zip -q -r inference_results_12ch.zip {OUTPUT_DIR}

/content/PointNeXt/openpoints/models/backbone/pointnetv2.py:79: SyntaxWarning: invalid escape sequence '\s'
  new_features: (B, \sum_k(mlps[k][-1], npoint)) tensor of the new_features descriptors
/content/PointNeXt/openpoints/models/layers/group.py:79: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/content/PointNeXt/openpoints/models/layers/upsampling.py:46: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/content/PointNeXt/openpoints/models/backbone/pointvit_inv.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/PointNeXt/openpoints/models/backbone/pointvit_inv.py:59: FutureWarn


🚀 Starting Inference using model: shapenetpart-train-custom_fallen_trees-ngpus1-seed7520-20251212-155340-E84zcZ3REWTnK2wbniefub_ckpt_best.pth


/tmp/ipython-input-750867153.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


✅ Model loaded.


  0%|          | 0/213 [00:00<?, ?it/s]/content/PointNeXt/openpoints/models/layers/subsample.py:93: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  output = torch.cuda.IntTensor(B, npoint)
100%|██████████| 213/213 [00:33<00:00,  6.35it/s]



Overall Accuracy (OA): 98.00%
Instance mIoU:       74.89%
Class mIoU:          86.97%
PER-PART BREAKDOWN:
Standing Trunk (0)  : 99.71%
Fallen Trunk (1)    : 97.17%
Root Ball (2)       : 75.46%
Pit (3)             : 75.56%


# Download Result

In [21]:
from google.colab import files
files.download('inference_results_12ch.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Test Using Official Code

In [22]:
# Patch main.py to fix Tensor Key Error
import os

target_file = "/content/PointNeXt/examples/shapenetpart/main.py"
print(f"Patching {target_file} to fix Tensor Key Error...")

with open(target_file, 'r') as f:
    code = f.read()

# The problematic line: parts = cls2parts[cls[shape_idx]]
# We change it to: parts = cls2parts[int(cls[shape_idx])]

bad_line = "parts = cls2parts[cls[shape_idx]]"
good_line = "parts = cls2parts[int(cls[shape_idx])]"

if bad_line in code:
    code = code.replace(bad_line, good_line)
    print("\u2705 Patched: Cast tensor to int for dictionary lookup.")
else:
    print("\u26A0\uFE0F Warning: Could not find exact line. Check if file changed.")

with open(target_file, 'w') as f:
    f.write(code)

Patching /content/PointNeXt/examples/shapenetpart/main.py to fix Tensor Key Error...
✅ Patched: Cast tensor to int for dictionary lookup.


In [23]:
import os
import glob

# 1. Find the Model in Drive
drive_model_dir = "/content/drive/MyDrive/ML_Projects/PointNeXt/Models"
model_candidates = glob.glob(os.path.join(drive_model_dir, "*_best.pth"))

if model_candidates:
    # Pick the latest one
    MODEL_PATH = max(model_candidates, key=os.path.getmtime)
    print(f"\u2705 Found Model: {os.path.basename(MODEL_PATH)}")
else:
    raise FileNotFoundError("\u274c No model found in Drive! Did you run the backup cell?")

# 2. Define Config Path
config_path = "/content/PointNeXt/cfgs/shapenetpart/custom_fallen_trees.yaml"

# 3. Run Official Evaluation
!PYTHONPATH=. python examples/shapenetpart/main.py --cfg {config_path} mode=test pretrained_path={MODEL_PATH} dataset.val.split=test

✅ Found Model: shapenetpart-train-custom_fallen_trees-ngpus1-seed7520-20251212-155340-E84zcZ3REWTnK2wbniefub_ckpt_best.pth
2025-12-12 16:54:09.997696: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765558450.296571    6722 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765558450.377996    6722 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765558450.994820    6722 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765558450.994879    6722 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid link